# Belief in Supernatural Evil and Guns


---

>Stogiannidis Ilias Marios <br />
>Department of Informatics  <br />
>Athens University of Economics and Business  <br />
>stoyianel@gmail.com

### Q1: Belief in Supernatural Evil Metric

#### Loading data from csv to python 

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("BaylorReligionSurvey.csv")


In order to carry out the study I have to find what data is needed. <br/>
After examining the data provided I came to the conclusion that the answers <br/> 
to the following questions they need

Ηere we see which question corresponds to the data we want

* Q73A = Ban on Semi-Auto Guns <br/>
* Q73E = Ban on High-Capacity Ammo Clips <br/>
* Q73G = Banning Civilian Handguns <br/>
* Q73H = Support for Concealed Carry Laws <br/>
* Q73C = More Armed Security at Schools <br/>
* Q73F = More Teachers/Faculty having Guns <br/> 
* Q73B = More Gun Safety Program <br/>
* Q73D = Expanded Mental Health Screening <br/>
* Q32 = Political Ideology <br/>
* Q4 = Attendance
* Q17 = Bible Literalism/Error 

Now we need to extract all the useful information to continue

In [2]:
spe_data = df[['Q73A','Q73E','Q73G','Q73H','Q73C','Q73D','Q73B','Q73F','Q4','Q17','RELTRAD','Q32',
'AGE','I_GENDER','Q88A','Q88B','Q88C','Q88D','Q88E','Q88F','Q89','Q90','Q95','Q51A','Q80','Q93','Q23A',
'Q23C','Q23G','STATE']].copy()

> Renaming the in order to be easier to work with 
> The metric I used is the average of the answers to the questions about hell, the Devil and demons <br/>

In [3]:
spe_data.rename(columns={'Q23A': 'Devil', 'Q23C': 'Hell','Q23G': 'Demons'},inplace= True)
spe_data["sn_metric"] = (spe_data["Devil"] + spe_data["Hell"] + spe_data["Demons"]) / 3

Here we see the correlation between the metric and the answers to the questions
As we can see the metric is highly correlated with them

In [4]:
spe_data[["Devil","Hell","Demons","sn_metric"]].corr().style.background_gradient(cmap="Blues")

,Devil,Hell,Demons,sn_metric
Devil,1.000000,0.885823,0.857858,0.963403
Hell,0.885823,1.000000,0.813177,0.947701
Demons,0.857858,0.813177,1.000000,0.937439
sn_metric,0.963403,0.947701,0.937439,1.000000


In [5]:
spe_data.drop(['Devil','Hell','Demons'],axis=1,inplace=True)


In [6]:
spe_data.describe()['sn_metric'].loc[['mean','std']]

mean    3.080280
std     1.056362
Name: sn_metric, dtype: float64

### Q2: Variables Selection

I continue renaming the data to be easier to work with <br/>
I also added the metric to the dataframe to be able to use it later <br/>
The modulo with 2 to the following columns is used to map 0 with opposed and 1 with support


In [7]:
spe_data.rename(columns={'Q73A':'Semi_Auto','Q73E':'High_Capacity','Q73G':'Handguns','Q73H':'Concealed_Carry','Q73C':'Armes_Security', 'Q73F':'Armed_Teachers', 'Q73B':'Gun_Safety','Q73D':'Mental_Health'}, inplace=True)
spe_data[['Semi_Auto','High_Capacity','Handguns','Concealed_Carry','Armes_Security','Gun_Safety','Mental_Health','Armed_Teachers']] %= 2


Renaming the columns in order to be easier to work with and to match the study <br/>
All renaming is done based on the way the data is presented in the csv file <br/>

Mapping state to South and Other in order to replicate the study<br/>

In [8]:
spe_data['STATE'] = spe_data['STATE'].map({'TX':'South','FL':'South','GA':'South','NC':'South','VA':'South','TN':'South','MD':'South','SC':'South',
'AL':'South','LA':'South','KY':'South','OK':'South','AR':'South','MS':'South','WV':'South','DE':'South'})
spe_data['STATE'].fillna('Other_Region',inplace=True)

Examining the data I came to the conclusion that the first to answers correspond to biblical literalism <br/>
and inerrancy. The other three are the questions about the Error <br/> 

In [9]:
spe_data.rename(columns={'Q4':'Attendance','Q17':'Bible_Literalism'}, inplace=True)
spe_data['Bible_Literalism'] =spe_data['Bible_Literalism'].map({1: 'literalism', 2: 'inerrancy', 3:'Error', 4:'Error', 8:'Error'})
spe_data = pd.get_dummies(spe_data)

Mapping the answers to the questions to the following values: <br/>
    1: Conservatist <br/>
    2: Mainline <br/>
    3: Black <br/>
    4: Catholic <br/>
    5, 6: Orthodox <br/>
    7: No affiliation <br/>

In [10]:
spe_data['RELTRAD'] = spe_data['RELTRAD'].map({1:'conservatist', 2:'mainline',
                                   3:'black',4:'catholic',5:'other',
                                   6:'other',7:'noaff'})

In [11]:
spe_data.rename(columns={'Q32':'Political_Views'}, inplace=True) 

> Mapping 0 to male and 1 to female 

In [12]:
spe_data['I_GENDER'] = spe_data['I_GENDER'] - 1 
spe_data['I_GENDER'] = spe_data['I_GENDER'].map({0.0:'Male',1.0:'Female'})

Finding which of the questioned people are white, black, hispanic and from other races 

In [13]:
spe_data['Q88A']=spe_data['Q88A'].map({1:'white',2:'other'})
spe_data['Q88B']=spe_data['Q88B'].map({1:'black',2:'noblack'})
spe_data['Q88C']=spe_data['Q88C'].map({1:'other',2:'null'})
spe_data['Q88D']=spe_data['Q88D'].map({1:'other',2:'null'})
spe_data['Q88E']=spe_data['Q88E'].map({1:'other',2:'null'})
spe_data['Q88F']=spe_data['Q88F'].map({1:'other',2:'null'})
spe_data['Q89']=spe_data['Q89'].map({1:'other',2:'hispanic',3:'hispanic',4:'hispanic',5:'hispanic',8:'other'})
spe_data = pd.get_dummies(spe_data)

Formating the data in order to be easier to work with 

In [14]:
spe_data['other'] = spe_data['Q88C_other'] + spe_data['Q88D_other'] + spe_data['Q88E_other']
+ spe_data['Q88F_other'] + spe_data['Q89_other'] + spe_data['Q88A_other'] 
spe_data.rename(columns= {'Q88A_white':'white', 'Q88B_black':'black', 'Q89_hispanic':'hispanic'},inplace=True)
spe_data.drop(['Q88A_other','Q88C_other','Q88D_other','Q88E_other','Q88F_other','Q89_other','Q88B_noblack','Q88C_null',
'Q88D_null','Q88F_null','Q88E_null'],axis=1,inplace=True)

Mapping the answers to the questions to the following values: <br/>
    1,2: Less Than High School <br/>
    3: High School <br/>
    4,5: Some college <br/>
    6: College Graduate <br/>
    7: Post-Graduate Degree <br/>

In [15]:
spe_data.rename(columns={'Q90':'Education'}, inplace=True)
spe_data['Education'] = spe_data['Education'].map({1:'Less Than High School',2:'Less Than High School',3:'High School',
                           4:'Some College', 5:'Some College', 6:'College Graduate', 7:'Post-graduate Degree'})


In [16]:
spe_data.rename(columns={'Q95':'Income'}, inplace=True)

Running through the answers in the question Q51A I made the assumption that only people that are married now <br/>
will be considered as married and those who are now divorced will be considered as not married <br/>

In [17]:
spe_data.rename(columns={'Q51A':'Marital_Status'}, inplace=True)
spe_data['Marital_Status'] = spe_data['Marital_Status'].map({1:'Not Partnered/Single',2:'Married/Cohabitating',3:"Married/Cohabitating",
                                   4:'Not Partnered/Single',5:'Not Partnered/Single',6:'Not Partnered/Single'})

In [18]:
spe_data.rename(columns={'Q80':'area'}, inplace=True)
spe_data['area'] = spe_data['area'].map({1:'Urban Area',2:'Urban Area',3:'Small Town/Rural',4:'Small Town/Rural'})

In [19]:
spe_data.rename(columns={'Q93':'Kids'}, inplace=True)
spe_data['Kids'].fillna(0,inplace=True)
spe_data['Kids'] = spe_data['Kids'].map({0:'No kids under 18 in home',1:'Kids under 18 in home',2:'Kids under 18 in home',3:"Kids under 18 in home", 
4:'Kids under 18 in home',5:'Kids under 18 in home',6:'Kids under 18 in home'})

Creating dummies for the above mentioned variables <br/>

In [20]:
spe_data= pd.get_dummies(spe_data)

Renaming the columns in order to present the statistics to match the study 

In [21]:
spe_data.rename(columns={'RELTRAD_black':'Black_Protestant', 'RELTRAD_catholic':'Catholic', 'RELTRAD_conservatist':'Conservative_Protestant','RELTRAD_mainline':'Mainline_Protestant',
'RELTRAD_other':'Other_Reg', 'RELTRAD_noaff':'No_Affiliation'}, inplace=True)
spe_data.rename(columns={'Marital_Status_Married/Cohabitating':'Married', 'Marital_Status_Not Partnered/Single':'Not Partnered/Single'}, inplace=True)
spe_data.rename(columns={'Education_Less Than High School':'Less_Than_High_School', 'Education_High School':'High_School', 'Education_Some College':'Some_College',
'Education_College Graduate':'College_Graduate', 'Education_Post-graduate Degree':'Postgraduate'}, inplace=True)
spe_data.rename(columns={'Kids_No kids under 18 in home':'No_Children', 'Kids_Kids under 18 in home':'Children'}, inplace=True) 
spe_data.rename(columns={'area_Urban Area':'Urban', 'area_Small Town/Rural':'Small Town/Rural'}, inplace=True)
spe_data.rename(columns={'Bible_Literalism_inerrancy':'Inerrancy', 'Bible_Literalism_Error':'Error', 'Bible_Literalism_literalism': 'Literalism'}, inplace=True)
spe_data.rename(columns={'I_GENDER_Male': 'Male', 'I_GENDER_Female': 'Female'}, inplace=True)
spe_data.rename(columns={'STATE_Other_Region': 'Other_Region', 'STATE_South': 'South'},inplace= True)
spe_data['AGE'].replace(0,np.NaN,inplace=True)
spe_data.describe().loc['mean']

Semi_Auto                   0.629161
High_Capacity               0.671105
Handguns                    0.290473
Concealed_Carry             0.557923
Armes_Security              0.621854
Mental_Health               0.908069
Gun_Safety                  0.927393
Armed_Teachers              0.294548
Attendance                  3.676735
Political_Views             4.284024
AGE                        52.645095
Income                      4.151329
sn_metric                   3.080280
Error                       0.426209
Inerrancy                   0.333333
Literalism                  0.225827
Other_Region                0.617048
South                       0.382952
Black_Protestant            0.067430
Catholic                    0.234733
Conservative_Protestant     0.267812
Mainline_Protestant         0.159033
No_Affiliation              0.136132
Other_Reg                   0.076972
Female                      0.565522
Male                        0.411578
white                       0.746183
b

### Q3: Predict Support for Various Gun Policies

Creating the strings in order to use them for training to train the models <br/>

In [22]:
semi_auto_ban = 'Semi_Auto ~ sn_metric+ Attendance + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
high_cap = 'High_Capacity ~ sn_metric + Attendance + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
handgun = 'Handguns ~ sn_metric + Attendance + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
concealed = 'Concealed_Carry ~ sn_metric + Attendance + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
armed_sec = 'Armes_Security ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
gun_safety = 'Gun_Safety ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
armed_teach = 'Armed_Teachers ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'
mental_health = 'Mental_Health ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg'

> I created a function that the summary of each regression model <br/>
> and converts the wanted data to a dataframe in order to be presented <br/> 

In [23]:
def summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher,
                               "odds_ratio":np.exp(coeff)
                                })

    #Reordering...
    results_df = results_df[["odds_ratio","coeff","pvals","conf_lower","conf_higher"]]
    results_df.drop(index=results_df.index[0], axis=0, inplace=True)
    return results_df

Following are all the logistic regression models for religious variables <br/>
and religious plus control variables <br/>


In [24]:
import statsmodels.formula.api as smf

In [25]:
logit = smf.logit(semi_auto_ban, data=spe_data).fit() 
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.644146
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.713088,-0.338150,0.000015,-0.491109,-0.185192
Attendance,1.022121,0.021880,0.390026,-0.028010,0.071770
Inerrancy,1.010619,0.010563,0.947166,-0.301850,0.322976
Literalism,0.959984,-0.040838,0.837493,-0.431094,0.349418
Mainline_Protestant,1.351892,0.301505,0.153651,-0.112680,0.715690
Black_Protestant,1.998134,0.692214,0.020934,0.104683,1.279744
Conservative_Protestant,1.176692,0.162707,0.466747,-0.275473,0.600888
Catholic,1.550478,0.438563,0.030610,0.041003,0.836124
Other_Reg,2.056762,0.721133,0.007549,0.192092,1.250173


In [26]:
logit = smf.logit(high_cap, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.620926
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.732690,-0.311032,0.000107,-0.468425,-0.153639
Attendance,1.029060,0.028646,0.271171,-0.022378,0.079670
Inerrancy,1.003269,0.003264,0.984102,-0.317796,0.324324
Literalism,0.926783,-0.076036,0.707204,-0.472794,0.320722
Mainline_Protestant,1.197354,0.180114,0.404772,-0.243610,0.603838
Black_Protestant,1.816821,0.597088,0.053672,-0.009436,1.203613
Conservative_Protestant,1.064072,0.062103,0.785015,-0.384103,0.508310
Catholic,1.394641,0.332637,0.109334,-0.074533,0.739807
Other_Reg,1.933603,0.659385,0.019477,0.106210,1.212561


In [27]:
logit = smf.logit(concealed, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.657706
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.512348,0.413663,3.455292e-08,0.266696,0.560630
Attendance,0.983456,-0.016683,5.104037e-01,-0.066359,0.032994
Inerrancy,0.776624,-0.252799,1.089653e-01,-0.561919,0.056321
Literalism,1.036601,0.035947,8.577250e-01,-0.357062,0.428956
Mainline_Protestant,1.106404,0.101115,6.248461e-01,-0.304171,0.506402
Black_Protestant,0.400700,-0.914542,1.580308e-03,-1.481866,-0.347218
Conservative_Protestant,1.072555,0.070044,7.520967e-01,-0.364567,0.504654
Catholic,0.681700,-0.383166,5.048321e-02,-0.767138,0.000807
Other_Reg,0.570195,-0.561777,2.354841e-02,-1.048022,-0.075532


In [28]:
logit = smf.logit(armed_sec, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.622458
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.599879,0.469928,4.657475e-10,0.322094,0.617762
Attendance,0.923933,-0.079116,3.387555e-03,-0.132035,-0.026196
Inerrancy,1.214748,0.194537,2.223861e-01,-0.117939,0.507013
Literalism,2.010521,0.698394,9.611627e-04,0.283806,1.112982
Mainline_Protestant,1.335712,0.289465,1.676716e-01,-0.121734,0.700663
Black_Protestant,1.499166,0.404909,2.020770e-01,-0.217207,1.027025
Conservative_Protestant,1.250720,0.223719,3.259822e-01,-0.222691,0.670130
Catholic,1.189780,0.173769,3.840072e-01,-0.217462,0.564999
Other_Reg,1.144587,0.135044,5.846598e-01,-0.349197,0.619285


In [29]:
logit = smf.logit(gun_safety, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.255263
         Iterations 7


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.831805,-0.184157,0.212829,-0.473878,0.105564
Attendance,0.955220,-0.045814,0.341095,-0.140134,0.048506
Inerrancy,1.080849,0.077747,0.798228,-0.518324,0.673817
Literalism,0.862645,-0.147752,0.681038,-0.852267,0.556762
Mainline_Protestant,1.496833,0.403352,0.307900,-0.371977,1.178680
Black_Protestant,1.751220,0.560313,0.298738,-0.496512,1.617138
Conservative_Protestant,1.531962,0.426549,0.299702,-0.379587,1.232685
Catholic,1.842164,0.610941,0.115684,-0.150221,1.372103
Other_Reg,1.790882,0.582708,0.250805,-0.411798,1.577214


In [30]:
logit = smf.logit(armed_teach, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.581347
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.481244,0.392882,0.000003,0.227050,0.558715
Attendance,0.956170,-0.044819,0.096120,-0.097611,0.007973
Inerrancy,0.880967,-0.126735,0.455787,-0.459795,0.206324
Literalism,0.856480,-0.154924,0.459251,-0.565210,0.255362
Mainline_Protestant,0.965367,-0.035247,0.876663,-0.480369,0.409874
Black_Protestant,0.515330,-0.662947,0.050252,-1.326622,0.000727
Conservative_Protestant,1.433369,0.360028,0.127403,-0.102863,0.822918
Catholic,0.784142,-0.243165,0.269977,-0.675210,0.188880
Other_Reg,0.805682,-0.216066,0.448920,-0.775327,0.343195


In [31]:
logit = smf.logit(mental_health, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.300110
         Iterations 7


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.799672,-0.223554,0.098775,-0.488971,0.041864
Attendance,0.989144,-0.010915,0.794694,-0.093126,0.071296
Inerrancy,1.179868,0.165403,0.545259,-0.370545,0.701350
Literalism,1.054851,0.053400,0.866944,-0.571291,0.678090
Mainline_Protestant,1.369308,0.314306,0.399486,-0.416850,1.045461
Black_Protestant,1.532188,0.426697,0.410954,-0.590445,1.443839
Conservative_Protestant,0.743739,-0.296065,0.419137,-1.014303,0.422172
Catholic,1.615751,0.479800,0.184800,-0.229329,1.188929
Other_Reg,1.483804,0.394609,0.401261,-0.526818,1.316036


In [32]:
semi_auto_ban2 = 'Semi_Auto ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
high_cap2 = 'High_Capacity ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
handgun2 = 'Handguns ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
concealed2 = 'Concealed_Carry ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic +  Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
armed_sec2 = 'Armes_Security ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
gun_safety2 = 'Gun_Safety ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
armed_teach2 = 'Armed_Teachers ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'
mental_health2 = 'Mental_Health ~ sn_metric + Attendance  + Inerrancy + Literalism + Mainline_Protestant + Black_Protestant + Conservative_Protestant + Catholic + Other_Reg + Political_Views + black + +hispanic + other + Female + Less_Than_High_School + Some_College + College_Graduate + Postgraduate + AGE + Income + Married + Children + Urban + South'

In [33]:
logit = smf.logit(semi_auto_ban2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.535211
         Iterations 6


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.739597,-0.301649,1.457160e-03,-0.487388,-0.115911
Attendance,1.036511,0.035860,2.601502e-01,-0.026557,0.098277
Inerrancy,1.121163,0.114366,5.542620e-01,-0.264665,0.493398
Literalism,1.222501,0.200899,4.131696e-01,-0.280270,0.682068
Mainline_Protestant,1.241301,0.216160,3.998084e-01,-0.287028,0.719347
Black_Protestant,0.529439,-0.635937,1.732834e-01,-1.551253,0.279380
Conservative_Protestant,1.330812,0.285789,2.890569e-01,-0.242545,0.814124
Catholic,1.410114,0.343670,1.595061e-01,-0.135156,0.822496
Other_Reg,1.564756,0.447730,1.621889e-01,-0.180090,1.075550
Political_Views,1.521565,0.419740,4.834531e-20,0.330003,0.509476


In [34]:
logit = smf.logit(high_cap2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.532221
         Iterations 6


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.791324,-0.234048,1.372387e-02,-0.420187,-0.047908
Attendance,1.035053,0.034453,2.786252e-01,-0.027874,0.096780
Inerrancy,1.068554,0.066307,7.316469e-01,-0.312652,0.445265
Literalism,1.045804,0.044786,8.545787e-01,-0.434145,0.523716
Mainline_Protestant,1.015160,0.015046,9.533332e-01,-0.488862,0.518954
Black_Protestant,0.423736,-0.858645,6.904644e-02,-1.784281,0.066991
Conservative_Protestant,0.959292,-0.041560,8.772074e-01,-0.568747,0.485627
Catholic,1.102058,0.097180,6.917863e-01,-0.383277,0.577636
Other_Reg,1.333612,0.287891,3.765765e-01,-0.350247,0.926029
Political_Views,1.454142,0.374416,2.402302e-16,0.284924,0.463908


In [35]:
logit = smf.logit(handgun2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.513164
         Iterations 6


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.711767,-0.340005,2.979245e-04,-0.524241,-0.155770
Attendance,1.063314,0.061390,6.651684e-02,-0.004183,0.126964
Inerrancy,1.005853,0.005836,9.767513e-01,-0.386686,0.398358
Literalism,1.378523,0.321012,2.175634e-01,-0.189252,0.831276
Mainline_Protestant,0.852764,-0.159272,5.439100e-01,-0.673626,0.355082
Black_Protestant,1.351216,0.301005,4.886971e-01,-0.551065,1.153075
Conservative_Protestant,0.838050,-0.176678,5.375132e-01,-0.738296,0.384941
Catholic,1.113742,0.107726,6.592096e-01,-0.371042,0.586493
Other_Reg,1.340265,0.292867,3.092663e-01,-0.271678,0.857412
Political_Views,1.401475,0.337525,4.308775e-14,0.249920,0.425131


In [36]:
logit = smf.logit(concealed2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.598240
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.491736,0.399941,3.890996e-06,0.230166,0.569715
Attendance,0.963659,-0.037017,2.029170e-01,-0.093998,0.019963
Inerrancy,0.682926,-0.381368,3.461832e-02,-0.735150,-0.027587
Literalism,0.790866,-0.234626,3.157451e-01,-0.693000,0.223748
Mainline_Protestant,1.236952,0.212651,3.212900e-01,-0.207578,0.632880
Black_Protestant,0.684419,-0.379185,3.374136e-01,-1.153914,0.395545
Conservative_Protestant,1.205448,0.186851,4.096797e-01,-0.257346,0.631049
Catholic,0.866187,-0.143654,4.704865e-01,-0.533791,0.246483
Political_Views,0.772182,-0.258535,1.141120e-10,-0.337133,-0.179937
black,1.172321,0.158985,5.896934e-01,-0.418832,0.736803


In [37]:
logit = smf.logit(armed_sec2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.596486
         Iterations 5


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.505948,0.409423,0.000001,0.243441,0.575404
Attendance,0.904587,-0.100277,0.001071,-0.160359,-0.040195
Inerrancy,1.133743,0.125525,0.474255,-0.218290,0.469340
Literalism,1.587544,0.462188,0.053139,-0.006255,0.930631
Mainline_Protestant,1.744512,0.556475,0.016927,0.099799,1.013150
Black_Protestant,1.358018,0.306027,0.486140,-0.555185,1.167238
Conservative_Protestant,1.311732,0.271349,0.280584,-0.221542,0.764240
Catholic,1.517910,0.417335,0.061024,-0.019301,0.853971
Other_Reg,1.527466,0.423610,0.122923,-0.114604,0.961825
Political_Views,0.878246,-0.129828,0.001045,-0.207453,-0.052203


In [38]:
logit = smf.logit(gun_safety2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.225821
         Iterations 8


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.990582,-0.009463,9.549045e-01,-0.337441,0.318515
Attendance,0.940560,-0.061280,2.701769e-01,-0.170205,0.047645
Inerrancy,1.279423,0.246409,4.757947e-01,-0.430865,0.923683
Literalism,1.151351,0.140936,7.307756e-01,-0.661838,0.943711
Mainline_Protestant,2.119995,0.751414,1.119203e-01,-0.175061,1.677889
Black_Protestant,0.684123,-0.379617,6.525729e-01,-2.032330,1.273095
Conservative_Protestant,1.735995,0.551581,2.261848e-01,-0.341692,1.444853
Catholic,1.820505,0.599114,1.622247e-01,-0.241053,1.439281
Other_Reg,1.683908,0.521117,3.522044e-01,-0.576751,1.618985
Political_Views,1.608098,0.475052,2.008566e-08,0.309121,0.640984


In [39]:
logit = smf.logit(armed_teach2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.529190
         Iterations 6


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,1.451670,0.372715,1.202028e-04,0.182761,0.562668
Attendance,0.944705,-0.056883,7.139755e-02,-0.118720,0.004954
Inerrancy,0.903009,-0.102023,5.950431e-01,-0.478212,0.274167
Literalism,0.712286,-0.339276,1.629368e-01,-0.815866,0.137315
Mainline_Protestant,0.979429,-0.020786,9.360873e-01,-0.528831,0.487259
Black_Protestant,1.766917,0.569236,2.521274e-01,-0.404991,1.543463
Conservative_Protestant,1.410490,0.343937,2.000377e-01,-0.182113,0.869987
Catholic,0.824814,-0.192598,4.424458e-01,-0.684069,0.298874
Other_Reg,0.948686,-0.052677,8.701122e-01,-0.684085,0.578730
Political_Views,0.764936,-0.267963,1.269411e-09,-0.354469,-0.181457


In [40]:
logit = smf.logit(mental_health2, data =spe_data).fit()   
var = summary_to_dataframe(logit) 
var

Optimization terminated successfully.
         Current function value: 0.273281
         Iterations 8


,odds_ratio,coeff,pvals,conf_lower,conf_higher
sn_metric,0.876123,-0.132249,0.384524,-0.430323,0.165825
Attendance,0.988174,-0.011896,0.806412,-0.107043,0.083251
Inerrancy,1.033652,0.033098,0.913956,-0.567283,0.633479
Literalism,0.899740,-0.105650,0.772806,-0.822882,0.611582
Mainline_Protestant,1.560531,0.445026,0.287348,-0.374777,1.264830
Black_Protestant,0.473825,-0.746918,0.344601,-2.295865,0.802030
Conservative_Protestant,0.958898,-0.041971,0.917751,-0.838555,0.754614
Catholic,1.659094,0.506272,0.207056,-0.280186,1.292730
Other_Reg,1.398761,0.335587,0.509564,-0.661716,1.332889
Political_Views,1.309489,0.269637,0.000131,0.131438,0.407836


As we can see the belief in the supernatural evil is highly associated with the support of people having guns, <br/>
wanting armed security and armed teachersS 

### Q4: Additional Estimations of the Strength of Predictors

In [41]:
def _summary_to_dataframe(results):
    summary_df = summary_to_dataframe(results)
    summary_df.drop(columns='odds_ratio', inplace=True)
    return summary_df


Below we the results that were produced by linear regression

In [42]:
lrg = smf.ols(semi_auto_ban, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.073790,0.000017,-0.107326,-0.040254
Attendance,0.005143,0.377307,-0.006280,0.016566
Inerrancy,0.001611,0.964519,-0.069435,0.072658
Literalism,-0.011619,0.800057,-0.101590,0.078353
Mainline_Protestant,0.065348,0.171608,-0.028375,0.159072
Black_Protestant,0.154172,0.022322,0.021954,0.286390
Conservative_Protestant,0.029627,0.561789,-0.070520,0.129774
Catholic,0.096083,0.034953,0.006795,0.185371
Other_Reg,0.146934,0.009277,0.036299,0.257568


In [43]:
lrg = smf.ols(high_cap, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.063959,0.000132,-0.096696,-0.031221
Attendance,0.006384,0.261409,-0.004763,0.017531
Inerrancy,-0.000339,0.992367,-0.069756,0.069079
Literalism,-0.019490,0.661544,-0.106802,0.067822
Mainline_Protestant,0.036016,0.439635,-0.055379,0.127412
Black_Protestant,0.125860,0.056244,-0.003355,0.255074
Conservative_Protestant,0.005889,0.905613,-0.091516,0.103294
Catholic,0.068490,0.122577,-0.018471,0.155450
Other_Reg,0.123357,0.025288,0.015298,0.231416


In [44]:
lrg = smf.ols(handgun, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.084998,8.175723e-08,-0.115923,-0.054072
Attendance,0.010036,6.183867e-02,-0.000498,0.020569
Inerrancy,-0.002622,9.376118e-01,-0.068330,0.063085
Literalism,0.048473,2.491793e-01,-0.034009,0.130954
Mainline_Protestant,-0.041952,3.396624e-01,-0.128111,0.044207
Black_Protestant,0.218838,4.413071e-04,0.096965,0.340711
Conservative_Protestant,-0.031797,4.973402e-01,-0.123679,0.060085
Catholic,0.073322,8.050448e-02,-0.008914,0.155559
Other_Reg,0.116939,2.560171e-02,0.014283,0.219596


In [45]:
lrg = smf.ols(concealed, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.099139,1.406994e-08,0.065056,0.133221
Attendance,-0.003960,5.024594e-01,-0.015540,0.007620
Inerrancy,-0.059692,1.063896e-01,-0.132167,0.012782
Literalism,0.005717,9.015878e-01,-0.084953,0.096386
Mainline_Protestant,0.024674,6.115574e-01,-0.070612,0.119961
Black_Protestant,-0.218511,1.392374e-03,-0.352354,-0.084669
Conservative_Protestant,0.016280,7.526577e-01,-0.085038,0.117597
Catholic,-0.090967,4.896729e-02,-0.181523,-0.000412
Other_Reg,-0.132737,2.186551e-02,-0.246182,-0.019292


In [46]:
lrg = smf.ols(armed_sec, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.110452,5.128458e-11,0.077716,0.143187
Attendance,-0.016754,3.205668e-03,-0.027885,-0.005622
Inerrancy,0.043260,2.228205e-01,-0.026321,0.112840
Literalism,0.140862,1.538906e-03,0.053789,0.227935
Mainline_Protestant,0.065154,1.633150e-01,-0.026483,0.156790
Black_Protestant,0.082582,2.083924e-01,-0.046132,0.211297
Conservative_Protestant,0.048446,3.308281e-01,-0.049247,0.146139
Catholic,0.038488,3.873860e-01,-0.048832,0.125808
Other_Reg,0.028660,6.042162e-01,-0.079778,0.137098


In [47]:
lrg = smf.ols(gun_safety, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.010898,0.238014,-0.029006,0.007211
Attendance,-0.002968,0.344410,-0.009122,0.003187
Inerrancy,0.005430,0.781272,-0.032927,0.043787
Literalism,-0.012468,0.611864,-0.060656,0.035721
Mainline_Protestant,0.024712,0.337423,-0.025805,0.075229
Black_Protestant,0.035770,0.323699,-0.035305,0.106844
Conservative_Protestant,0.025468,0.353802,-0.028394,0.079330
Catholic,0.037350,0.128515,-0.010824,0.085524
Other_Reg,0.034232,0.261307,-0.025523,0.093986


In [48]:
lrg = smf.ols(armed_teach, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.073639,0.000004,0.042350,0.104928
Attendance,-0.009215,0.090485,-0.019886,0.001456
Inerrancy,-0.023583,0.486174,-0.089993,0.042828
Literalism,-0.028471,0.503706,-0.111975,0.055032
Mainline_Protestant,-0.004957,0.911267,-0.092199,0.082285
Black_Protestant,-0.120480,0.055284,-0.243687,0.002727
Conservative_Protestant,0.086285,0.069480,-0.006886,0.179456
Catholic,-0.044365,0.296474,-0.127692,0.038963
Other_Reg,-0.034533,0.512618,-0.137966,0.068900


In [49]:
lrg = smf.ols(mental_health, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.015745,0.126722,-0.035957,0.004468
Attendance,-0.000915,0.794131,-0.007788,0.005959
Inerrancy,0.012966,0.552984,-0.029893,0.055824
Literalism,0.000892,0.974048,-0.052891,0.054676
Mainline_Protestant,0.022823,0.427168,-0.033544,0.079190
Black_Protestant,0.032846,0.418574,-0.046785,0.112478
Conservative_Protestant,-0.036845,0.230128,-0.097049,0.023358
Catholic,0.033578,0.220889,-0.020205,0.087361
Other_Reg,0.026685,0.433912,-0.040191,0.093562


In [50]:
lrg = smf.ols(semi_auto_ban2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.052142,2.174504e-03,-0.085444,-0.018839
Attendance,0.006606,2.557144e-01,-0.004792,0.018004
Inerrancy,0.018596,5.949268e-01,-0.050001,0.087193
Literalism,0.031217,4.936186e-01,-0.058220,0.120654
Mainline_Protestant,0.035116,4.484417e-01,-0.055741,0.125972
Black_Protestant,-0.105738,2.025520e-01,-0.268442,0.056965
Conservative_Protestant,0.042410,3.919793e-01,-0.054752,0.139573
Catholic,0.060419,1.748582e-01,-0.026897,0.147735
Other_Reg,0.052597,3.330478e-01,-0.053962,0.159155
Political_Views,0.074872,3.213335e-21,0.059619,0.090126


In [51]:
lrg = smf.ols(high_cap2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.040999,1.538926e-02,-0.074148,-0.007850
Attendance,0.006556,2.561089e-01,-0.004765,0.017877
Inerrancy,0.010674,7.591627e-01,-0.057617,0.078965
Literalism,0.002800,9.504966e-01,-0.085678,0.091279
Mainline_Protestant,0.000301,9.947800e-01,-0.090033,0.090635
Black_Protestant,-0.147410,7.493994e-02,-0.309669,0.014850
Conservative_Protestant,-0.019980,6.841997e-01,-0.116328,0.076369
Catholic,0.014869,7.365804e-01,-0.071832,0.101570
Other_Reg,0.026033,6.299141e-01,-0.079935,0.132000
Political_Views,0.065233,7.015356e-17,0.050117,0.080350


In [52]:
lrg = smf.ols(handgun2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.060180,2.731917e-04,-0.092528,-0.027832
Attendance,0.009519,9.146038e-02,-0.001537,0.020575
Inerrancy,0.003957,9.075592e-01,-0.062879,0.070792
Literalism,0.062817,1.534468e-01,-0.023465,0.149099
Mainline_Protestant,-0.023271,6.040319e-01,-0.111280,0.064738
Black_Protestant,0.079236,3.254430e-01,-0.078787,0.237259
Conservative_Protestant,-0.017753,7.104464e-01,-0.111543,0.076038
Catholic,0.024701,5.666208e-01,-0.059845,0.109247
Other_Reg,0.067102,2.059658e-01,-0.036933,0.171137
Political_Views,0.059083,8.409132e-15,0.044331,0.073836


In [53]:
lrg = smf.ols(concealed2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.084298,3.435606e-06,0.048838,0.119757
Attendance,-0.007174,2.329503e-01,-0.018969,0.004620
Inerrancy,-0.077720,3.815381e-02,-0.151189,-0.004252
Literalism,-0.051027,2.894297e-01,-0.145483,0.043430
Mainline_Protestant,0.040952,3.611216e-01,-0.046992,0.128896
Black_Protestant,-0.090936,2.843552e-01,-0.257505,0.075634
Conservative_Protestant,0.034217,4.674436e-01,-0.058137,0.126571
Catholic,-0.035481,3.993945e-01,-0.118054,0.047092
Political_Views,-0.054594,6.154036e-11,-0.070828,-0.038361
black,0.035147,5.807300e-01,-0.089664,0.159957


In [54]:
lrg = smf.ols(armed_sec2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.091656,3.944757e-07,0.056389,0.126922
Attendance,-0.019949,1.180052e-03,-0.031986,-0.007911
Inerrancy,0.025475,4.919380e-01,-0.047228,0.098177
Literalism,0.085933,7.260373e-02,-0.007893,0.179759
Mainline_Protestant,0.116555,1.764822e-02,0.020318,0.212791
Black_Protestant,0.061499,4.836306e-01,-0.110694,0.233692
Conservative_Protestant,0.057023,2.761627e-01,-0.045663,0.159709
Catholic,0.088089,6.205181e-02,-0.004446,0.180624
Other_Reg,0.086996,1.314908e-01,-0.026090,0.200083
Political_Views,-0.026873,1.071110e-03,-0.042952,-0.010793


In [55]:
lrg = smf.ols(gun_safety2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.000755,9.391365e-01,-0.020143,0.018634
Attendance,-0.003558,2.918238e-01,-0.010177,0.003061
Inerrancy,0.017531,3.889409e-01,-0.022376,0.057438
Literalism,0.004060,8.772587e-01,-0.047501,0.055621
Mainline_Protestant,0.037590,1.631400e-01,-0.015259,0.090439
Black_Protestant,-0.014882,7.573717e-01,-0.109365,0.079602
Conservative_Protestant,0.025227,3.792133e-01,-0.031035,0.081489
Catholic,0.029269,2.575759e-01,-0.021427,0.079964
Other_Reg,0.019373,5.397628e-01,-0.042594,0.081340
Political_Views,0.025051,3.053092e-08,0.016233,0.033868


In [56]:
lrg = smf.ols(armed_teach2 , data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,0.064356,1.296293e-04,0.031469,0.097243
Attendance,-0.010893,5.761533e-02,-0.022138,0.000353
Inerrancy,-0.017460,6.138701e-01,-0.085334,0.050413
Literalism,-0.057358,2.000744e-01,-0.145132,0.030416
Mainline_Protestant,-0.003445,9.398808e-01,-0.093036,0.086146
Black_Protestant,0.099784,2.234602e-01,-0.060942,0.260509
Conservative_Protestant,0.079226,1.041753e-01,-0.016357,0.174809
Catholic,-0.032397,4.608823e-01,-0.118563,0.053769
Other_Reg,0.006624,9.017239e-01,-0.098596,0.111845
Political_Views,-0.047351,7.715592e-10,-0.062337,-0.032364


In [57]:
lrg = smf.ols(mental_health2, data =spe_data).fit()
ddf = _summary_to_dataframe(lrg)
ddf

,coeff,pvals,conf_lower,conf_higher
sn_metric,-0.008928,0.419841,-0.030635,0.012778
Attendance,-0.001148,0.761507,-0.008570,0.006273
Inerrancy,0.004374,0.848065,-0.040404,0.049151
Literalism,-0.014317,0.626493,-0.072020,0.043385
Mainline_Protestant,0.029987,0.320105,-0.029161,0.089136
Black_Protestant,-0.047452,0.380598,-0.153589,0.058685
Conservative_Protestant,-0.016774,0.602315,-0.079913,0.046365
Catholic,0.030563,0.291882,-0.026301,0.087427
Other_Reg,0.016707,0.637976,-0.052936,0.086350
Political_Views,0.018630,0.000227,0.008746,0.028514


### Q5: Compare with Decision Trees / Random Forest Estimators

In [58]:
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


> Helper function to present the importances into a dataframe

In [59]:
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

In [60]:
y, X = dmatrices(semi_auto_ban2, spe_data, return_type='dataframe')

In [61]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

#AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X, y)
print(ada.score(X, y))

#Random Forest Classifier
forest = RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_split=2)
forest.fit(X, y)
print(forest.score(X, y))

#Extra Trees Classifier
et = ExtraTreesClassifier(n_estimators=100,max_depth=10,min_samples_split=2)
et.fit(X, y)
print(et.score(X, y))

#Decision Tree Classifier
dt = DecisionTreeClassifier(max_depth=10,min_samples_split=2)
dt.fit(X, y)
print(dt.score(X, y))


C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7291338582677165


C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


0.9669291338582677


C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.9488188976377953
0.9283464566929134


* We can see that random forest gives the best score for the subset so for <br/>
the rest of data I will be using just this

In [62]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,Political_Views,0.169029
1,AGE,0.153955
2,Female,0.089088
3,Income,0.078722
4,Attendance,0.078353
5,sn_metric,0.072611
6,Urban,0.034554
7,South,0.031070
8,Married,0.025787
9,Inerrancy,0.024747


In [63]:
y, X = dmatrices(high_cap2 , spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [64]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.168458
1,Political_Views,0.163362
2,Income,0.087153
3,Attendance,0.079267
4,Female,0.072598
5,sn_metric,0.071844
6,Urban,0.033812
7,Children,0.026071
8,Married,0.026065
9,South,0.025403


In [65]:
y, X = dmatrices(handgun2, spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [66]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.154269
1,Political_Views,0.151380
2,Income,0.092479
3,sn_metric,0.090025
4,Attendance,0.082598
5,Urban,0.036268
6,South,0.033020
7,Female,0.029408
8,hispanic,0.029365
9,Married,0.027640


In [67]:
y, X = dmatrices(concealed2, spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [68]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.143412
1,Political_Views,0.132743
2,sn_metric,0.097926
3,Income,0.089613
4,Attendance,0.084474
5,Female,0.047872
6,Urban,0.037171
7,South,0.035049
8,Married,0.034595
9,Children,0.030015


In [69]:
y, X = dmatrices(armed_sec2 , spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [70]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.156610
1,sn_metric,0.144156
2,Political_Views,0.094529
3,Income,0.092883
4,Attendance,0.087227
5,Postgraduate,0.039936
6,Urban,0.035743
7,South,0.034196
8,Female,0.029902
9,Married,0.027490


In [71]:
y, X = dmatrices(gun_safety2 , spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [72]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.177375
1,Political_Views,0.121648
2,Attendance,0.098723
3,Income,0.095376
4,sn_metric,0.074943
5,Urban,0.039099
6,Female,0.033897
7,South,0.032837
8,Married,0.029587
9,Children,0.027607


In [73]:
y, X = dmatrices(armed_teach2  , spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [74]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.165740
1,Political_Views,0.146680
2,Income,0.085171
3,Attendance,0.084454
4,sn_metric,0.081511
5,Female,0.048740
6,Urban,0.042193
7,Conservative_Protestant,0.032623
8,Married,0.031773
9,South,0.028911


In [75]:
y, X = dmatrices(mental_health2 , spe_data, return_type='dataframe')
forest.fit(X, y)

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=10)

In [76]:
importances = forest.feature_importances_
imdf = imp_df(X.columns, importances)
imdf

,feature,feature_importance
0,AGE,0.173644
1,Political_Views,0.112312
2,Income,0.099151
3,Attendance,0.091540
4,sn_metric,0.085996
5,Female,0.044226
6,South,0.036203
7,Urban,0.034649
8,Married,0.033997
9,Children,0.028937


As we can see the best predictors are: Age and one's Political Ideology. <br>
However the belief in supernatural evil is also a good predictor for some views <br>
about the possession of weapons 